## Notebook description:

Looking at a spinup metric - between most spunup runs and the previous one, what is the percent difference?

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [4]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
#from mpl_toolkits.basemap import Basemap
#from scipy.io import loadmat
import pickle
from cmocean import cm
import warnings
import netCDF4 as nc

import importlib
from salishsea_tools import viz_tools

%matplotlib inline
warnings.simplefilter('ignore')



#### MEANS EXTRACTING CODE  - uses extract_bylayer_mean.py, which is in this folder
    


In [5]:
w = nc.Dataset('/data/tjarniko/results/BASERUN_EXP/PI5/ncs/SKOG_1d_20150101_20150130_carp_T_20150101-20150101.nc')
print(w['dissolved_inorganic_carbon'])

<class 'netCDF4._netCDF4.Variable'>
float32 dissolved_inorganic_carbon(time_counter, deptht, y, x)
    standard_name: mole_concentration_of_dissolved_inorganic_carbon_in_sea_water
    long_name: Dissolved Inorganic C Concentration
    units: mmol m-3
    online_operation: average
    interval_operation: 40 s
    interval_write: 1 d
    cell_methods: time: mean (interval: 40 s)
    cell_measures: area: area
    _FillValue: 1e+20
    missing_value: 1e+20
    coordinates: time_centered deptht nav_lat nav_lon
unlimited dimensions: time_counter
current shape = (1, 40, 898, 398)
filling on


In [7]:
def extractor(start, end, ftype, sdir, varname, fname,  inletmask = False):

    '''USAGE:
    
    for a given variable in raw ncs of the PILA experiment 
    found in directory tree /data/tjarniko/results/BASERUN_EXP
    take by-layer means of the variable throughout the timeperiod specified

    start = '2015-01-01' #start of timeperiod
    end = '2015-12-31' #end of timeperiod (typically a year)
    ftype = 'carp' #type of model result .nc 
    sdir = 'MAIN/BR_3rd_2015' #where under directory tree do we find ncs 
    inletmask = True #are we masking out Toba/Bute/Jervis?
    varname = 'dissolved_inorganic_carbon' #name of variable
    fname = 'BR3_DIC_means' #name of resulting pkl 
    
    import extract_bylayer_mean as ebm
    ebm.extractor(start,end,ftype,sdir, inletmask, varname, fname)
    
    '''
    import matplotlib.pyplot as plt
    import netCDF4 as nc
    import numpy as np
    import scipy as sp
    import datetime as dt
    ""
    from salishsea_tools import (
        nc_tools,
        viz_tools,
        geo_tools,
        tidetools
    )
    import netCDF4 as nc
    import cmocean as cm
    import glob
    import arrow
    import gsw
    #import gef
    import pickle

    #where to store
    daily_means = np.zeros((40,365))
    ncfile_ar = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)
    #no of days in array
    dayslen = len(arrow_array)
    
    for i in range(0,dayslen):
        tdate = arrow_array[i][0]
        ddmmmyy = tdate.format('DDMMMYY').lower()
        ymd = tdate.format('YYYYMMDD')
        nc_ncfile = '/data/tjarniko/results/BASERUN_EXP/' + sdir + '/ncs/SKOG_1d_*'+ ftype +'*' + ymd + '-' + ymd + '.nc'
        tnc_ncfile = glob.glob(nc_ncfile)
        #print(tnc_ncfile[0])
        ncfile_ar.append(tnc_ncfile[0])
        if i%50 == 0:
            print(i)

    print('done making nclen')

    for i in range(0,dayslen):

        if (i%50 ==0):
            print(ncfile_ar[i])
        t_test = nc.Dataset(ncfile_ar[i])
        
        tdat = t_test[varname][:]
        if i==0:
            print('shape of dataset is')
            print(np.shape(tdat))
        #remove border - 20 grid cells north and west

        tdat[:,:,878:898,:] = np.nan
        tdat[:,:,:,0:20] = np.nan
        #no zeros
        tdat[tdat == 0] = np.nan
        
        if (inletmask == True):
            #mask out inlets
            tdat[0,:,700:898,200:398] = np.nan
            tdat[0,:,550:700,255:398] = np.nan

        if (i==0):
            print('nansum(dataset) -checksum to make sure inlet mask works - false/true should give different answers')
            print(np.nansum(tdat))
        tdat_fc = tdat[0,:,:,:]
        tdat_alldomain = np.zeros([40])
        for q in range(0,40):
            tdat_alldomain[q] = np.nanmean(tdat_fc[q,:,:])

        daily_means[:,i] =  tdat_alldomain

    fname =  fname + '.pkl'
    pickle.dump(daily_means, open(fname, 'wb'))
    
    return

In [8]:
import sys
sys.path.append('/data/tjarniko/MEOPAR/analysis_tereza/notebooks/PI_CARBON_PAPER/MAIN_ANALYSIS/pyscripts/bylayer_extract')

#     for a given variable in raw ncs of the PILA experiment 
#     found in directory tree /data/tjarniko/results/BASERUN_EXP
#     take by-layer means of the variable throughout the timeperiod specified

tstr = '/data/tjarniko/MEOPAR/analysis_tereza/notebooks/PI_CARBON_PAPER/MAIN_ANALYSIS/pkls/'
start = '2015-01-01' #start of timeperiod
end = '2015-12-31' #end of timeperiod (typically a year)
ftype = 'carp' #type of model result .nc 
sdir = 'PI8' #where under directory tree do we find ncs 
inletmask = True #are we masking out Toba/Bute/Jervis?
varname = 'dissolved_inorganic_carbon' #name of variable
fname = tstr + 'PI8_DIC_means' #name of resulting pkl 

extractor(start,end,ftype,sdir, varname, fname, inletmask )

0
50
100
150
200
250
300
350
done making nclen
/data/tjarniko/results/BASERUN_EXP/PI8/ncs/SKOG_1d_20150101_20150130_carp_T_20150101-20150101.nc
shape of dataset is
(1, 40, 898, 398)
nansum(dataset) -checksum to make sure inlet mask works - false/true should give different answers
3882585000.0
/data/tjarniko/results/BASERUN_EXP/PI8/ncs/SKOG_1d_20150131_20150301_carp_T_20150220-20150220.nc
/data/tjarniko/results/BASERUN_EXP/PI8/ncs/SKOG_1d_20150401_20150430_carp_T_20150411-20150411.nc
/data/tjarniko/results/BASERUN_EXP/PI8/ncs/SKOG_1d_20150531_20150629_carp_T_20150531-20150531.nc
/data/tjarniko/results/BASERUN_EXP/PI8/ncs/SKOG_1d_20150630_20150729_carp_T_20150720-20150720.nc
/data/tjarniko/results/BASERUN_EXP/PI8/ncs/SKOG_1d_20150829_20150927_carp_T_20150908-20150908.nc
/data/tjarniko/results/BASERUN_EXP/PI8/ncs/SKOG_1d_20151028_20151126_carp_T_20151028-20151028.nc
/data/tjarniko/results/BASERUN_EXP/PI8/ncs/SKOG_1d_20151127_20151231_carp_T_20151217-20151217.nc


In [9]:
import sys
sys.path.append('/data/tjarniko/MEOPAR/analysis_tereza/notebooks/PI_CARBON_PAPER/MAIN_ANALYSIS/pyscripts/bylayer_extract')

#     for a given variable in raw ncs of the PILA experiment 
#     found in directory tree /data/tjarniko/results/BASERUN_EXP
#     take by-layer means of the variable throughout the timeperiod specified

tstr = '/data/tjarniko/MEOPAR/analysis_tereza/notebooks/PI_CARBON_PAPER/MAIN_ANALYSIS/pkls/'
start = '2015-01-01' #start of timeperiod
end = '2015-12-31' #end of timeperiod (typically a year)
ftype = 'carp' #type of model result .nc 
sdir = 'PI9' #where under directory tree do we find ncs 
inletmask = True #are we masking out Toba/Bute/Jervis?
varname = 'dissolved_inorganic_carbon' #name of variable
fname = tstr + 'PI9_DIC_means' #name of resulting pkl 

# import extract_bylayer_mean as ebm
extractor(start,end,ftype,sdir, varname, fname, inletmask )

0
50
100
150
200
250
300
350
done making nclen
/data/tjarniko/results/BASERUN_EXP/PI9/ncs/SKOG_1d_20150101_20150130_carp_T_20150101-20150101.nc
shape of dataset is
(1, 40, 898, 398)
nansum(dataset) -checksum to make sure inlet mask works - false/true should give different answers
3886441500.0
/data/tjarniko/results/BASERUN_EXP/PI9/ncs/SKOG_1d_20150131_20150301_carp_T_20150220-20150220.nc
/data/tjarniko/results/BASERUN_EXP/PI9/ncs/SKOG_1d_20150401_20150430_carp_T_20150411-20150411.nc
/data/tjarniko/results/BASERUN_EXP/PI9/ncs/SKOG_1d_20150531_20150629_carp_T_20150531-20150531.nc
/data/tjarniko/results/BASERUN_EXP/PI9/ncs/SKOG_1d_20150630_20150729_carp_T_20150720-20150720.nc
/data/tjarniko/results/BASERUN_EXP/PI9/ncs/SKOG_1d_20150829_20150927_carp_T_20150908-20150908.nc
/data/tjarniko/results/BASERUN_EXP/PI9/ncs/SKOG_1d_20151028_20151126_carp_T_20151028-20151028.nc
/data/tjarniko/results/BASERUN_EXP/PI9/ncs/SKOG_1d_20151127_20151231_carp_T_20151217-20151217.nc


In [7]:
mesh = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')

In [8]:

#dic
BR1_DIC = pickle.load(open('../pkls/BR1_DIC_means_inletsIN.pkl' , 'rb')) 
BR2_DIC = pickle.load(open('../pkls/BR2_DIC_means_inletsIN.pkl' , 'rb')) 
BR3_DIC = pickle.load(open('../pkls/BR3_DIC_means_inletsIN.pkl' , 'rb')) 
PI1_DIC = pickle.load(open('../pkls/PI1_DIC_means_inletsIN.pkl' , 'rb')) 
PI2_DIC = pickle.load(open('../pkls/PI2_DIC_means_inletsIN.pkl' , 'rb')) 
PI3rr_DIC = pickle.load(open('../pkls/PI3rr_DIC_means_inletsIN.pkl' , 'rb')) 
PI4_DIC = pickle.load(open('../pkls/PI4_DIC_means_inletsIN.pkl' , 'rb')) 
PI4_DIC[PI4_DIC == 0] = np.nan

#ta
BR1_TA = pickle.load(open('../pkls/BR1_TA_means_inletsIN.pkl' , 'rb')) 
BR2_TA = pickle.load(open('../pkls/BR2_TA_means_inletsIN.pkl' , 'rb')) 
BR3_TA = pickle.load(open('../pkls/BR3_TA_means_inletsIN.pkl' , 'rb')) 
PI1_TA = pickle.load(open('../pkls/PI1_TA_means_inletsIN.pkl' , 'rb')) 
PI2_TA = pickle.load(open('../pkls/PI2_TA_means_inletsIN.pkl' , 'rb')) 
PI3rr_TA = pickle.load(open('../pkls/PI3rr_TA_means_inletsIN.pkl' , 'rb')) 
PI4_TA = pickle.load(open('../pkls/PI4_TA_means_inletsIN.pkl' , 'rb')) 
PI4_TA[PI4_TA == 0] = np.nan
#sal
BR1_sal = pickle.load(open('../pkls/BR1_sal_means_inletsIN.pkl' , 'rb')) 
BR2_sal = pickle.load(open('../pkls/BR2_sal_means_inletsIN.pkl' , 'rb')) 
BR3_sal = pickle.load(open('../pkls/BR3_sal_means_inletsIN.pkl' , 'rb')) 
PI1_sal = pickle.load(open('../pkls/PI1_sal_means_inletsIN.pkl' , 'rb')) 
PI2_sal = pickle.load(open('../pkls/PI2_sal_means_inletsIN.pkl' , 'rb')) 
PI3rr_sal = pickle.load(open('../pkls/PI3rr_sal_means_inletsIN.pkl' , 'rb')) 
PI4_sal = pickle.load(open('../pkls/PI4_sal_means_inletsIN.pkl' , 'rb')) 
PI4_sal[PI4_sal == 0] = np.nan
#temp
BR1_temp = pickle.load(open('../pkls/BR1_temp_means_inletsIN.pkl' , 'rb')) 
BR2_temp = pickle.load(open('../pkls/BR2_temp_means_inletsIN.pkl' , 'rb')) 
BR3_temp = pickle.load(open('../pkls/BR3_temp_means_inletsIN.pkl' , 'rb')) 
PI1_temp = pickle.load(open('../pkls/PI1_temp_means_inletsIN.pkl' , 'rb')) 
PI2_temp = pickle.load(open('../pkls/PI2_temp_means_inletsIN.pkl' , 'rb')) 
PI3rr_temp = pickle.load(open('../pkls/PI3rr_temp_means_inletsIN.pkl' , 'rb')) 
# PI4_temp = pickle.load(open('../pkls/PI4_temp_means_inletsIN.pkl' , 'rb')) 
# PI4_temp[PI4_temp == 0] = np.nan
#nitrate
BR1_nit = pickle.load(open('../pkls/BR1_nit_means_inletsIN.pkl' , 'rb')) 
BR2_nit = pickle.load(open('../pkls/BR2_nit_means_inletsIN.pkl' , 'rb')) 
BR3_nit = pickle.load(open('../pkls/BR3_nit_means_inletsIN.pkl' , 'rb')) 
PI1_nit = pickle.load(open('../pkls/PI1_nit_means_inletsIN.pkl' , 'rb')) 
PI2_nit = pickle.load(open('../pkls/PI2_nit_means_inletsIN.pkl' , 'rb')) 
PI3rr_nit = pickle.load(open('../pkls/PI3rr_nit_means_inletsIN.pkl' , 'rb')) 
PI4_nit = pickle.load(open('../pkls/PI4_nit_means_inletsIN.pkl' , 'rb')) 
PI4_nit[PI4_nit == 0] = np.nan
#diatoms
BR1_diat = pickle.load(open('../pkls/BR1_diat_means_inletsIN.pkl' , 'rb')) 
BR2_diat = pickle.load(open('../pkls/BR2_diat_means_inletsIN.pkl' , 'rb')) 
BR3_diat = pickle.load(open('../pkls/BR3_diat_means_inletsIN.pkl' , 'rb')) 
PI1_diat = pickle.load(open('../pkls/PI1_diat_means_inletsIN.pkl' , 'rb')) 
PI2_diat = pickle.load(open('../pkls/PI2_diat_means_inletsIN.pkl' , 'rb')) 
PI3rr_diat = pickle.load(open('../pkls/PI3rr_diat_means_inletsIN.pkl' , 'rb')) 
PI4_diat = pickle.load(open('../pkls/PI4_diat_means_inletsIN.pkl' , 'rb')) 
PI4_diat[PI4_diat == 0] = np.nan

##PI5 for all 
PI5_DIC = pickle.load(open('../pkls/PI5_DIC_means_inletsIN.pkl' , 'rb')) 
PI5_DIC[PI5_DIC == 0] = np.nan
PI5_TA = pickle.load(open('../pkls/PI5_TA_means_inletsIN.pkl' , 'rb')) 
PI5_TA[PI5_TA == 0] = np.nan
PI5_sal = pickle.load(open('../pkls/PI5_sal_means_inletsIN.pkl' , 'rb')) 
PI5_sal[PI5_sal == 0] = np.nan
PI5_temp = pickle.load(open('../pkls/PI5_temp_means_inletsIN.pkl' , 'rb')) 
PI5_temp[PI5_temp == 0] = np.nan
PI5_nit = pickle.load(open('../pkls/PI5_nit_means_inletsIN.pkl' , 'rb')) 
PI5_nit[PI5_nit == 0] = np.nan
PI5_diat = pickle.load(open('../pkls/PI5_diat_means_inletsIN.pkl' , 'rb')) 
PI5_diat[PI5_diat == 0] = np.nan

In [ ]:
    fig = plt.figure(figsize=(14*fact, 7*fact))

    ##BR - units of quantity
    BR_2ndspin = BR_varperc[:,365:365*2]
    PI_3rdspin = PI_varperc[:,365*3:365*4]

    ax1 = plt.subplot2grid((1,2), (0,0), colspan=1, rowspan=1)
    zdepths = np.zeros(40); t_means = np.zeros(40); t_stds = np.zeros(40); t_maxs = np.zeros(40); t_mins = np.zeros(40);
    for i in range(0,40):
        zdepths[i] = np.sum(depth[0:i+1]) 
        t_means[i] =   np.nanmean(BR_2ndspin[i,:])
        t_stds[i] = np.nanstd(BR_2ndspin[i,:])
        t_maxs[i] = np.nanmax(BR_2ndspin[i,:])
        t_mins[i] = np.nanmin(BR_2ndspin[i,:])
    print('mean diff by layer BR')
    print(t_means)
    ax1.plot(t_means,zdepths, marker = 'o', color = 'r', label = 'mean', markersize = 10)
    ax1.plot(t_means-t_stds, zdepths, linestyle = ':',  label = 'mean +/- std. dev', color = 'r', markersize = 8)
    ax1.plot(t_means+t_stds, zdepths, linestyle = ':', color = 'r', markersize = 8)
    ax1.plot(t_maxs, zdepths, marker = '*', label = 'max/min', linestyle = 'None' ,  color = 'r', markersize = 10)
    ax1.plot(t_mins, zdepths, marker = '*', linestyle = 'None' ,  color = 'r', markersize = 10)

    ax1.fill_betweenx(zdepths, t_means-t_stds, t_means+t_stds, color = 'r', alpha = 0.2)
    ax1.invert_yaxis()
    ax1.legend(loc = 'best', fontsize = 15)
    ax1.set_ylabel('depth', fontsize = 15)
    ax1.set_xlabel('%difference of by-layer \n mean of tracer between runs', fontsize = 15)
    ax1.set_title('present-day run - last spinup run', fontsize = 15)

    ax2 = plt.subplot2grid((1,2), (0,1), colspan=1, rowspan=1)
    zdepths = np.zeros(40); t_means = np.zeros(40); t_stds = np.zeros(40); t_maxs = np.zeros(40); t_mins = np.zeros(40);
    for i in range(0,40):
        zdepths[i] = np.sum(depth[0:i+1]) 
        t_means[i] =   np.nanmean(PI_3rdspin[i,:])
        t_stds[i] = np.nanstd(PI_3rdspin[i,:])
        t_maxs[i] = np.nanmax(PI_3rdspin[i,:])
        t_mins[i] = np.nanmin(PI_3rdspin[i,:])
    print('mean diff by layer PI')
    print(t_means)
    ax2.plot(t_means,zdepths, marker = 'o', color = 'cornflowerblue', markersize = 10)
    ax2.plot(t_means-t_stds, zdepths, linestyle = ':',  color = 'cornflowerblue', markersize = 8)
    ax2.plot(t_means+t_stds, zdepths, linestyle = ':', color = 'cornflowerblue', markersize = 8)
    ax2.plot(t_maxs, zdepths, marker = '*', linestyle = 'None' ,  color = 'cornflowerblue', markersize = 10)
    ax2.plot(t_mins, zdepths, marker = '*', linestyle = 'None' ,  color = 'cornflowerblue', markersize = 10)
    ax2.fill_betweenx(zdepths, t_means-t_stds, t_means+t_stds, color = 'cornflowerblue', alpha = 0.2)
    ax2.set_ylabel('depth', fontsize = 12)
    ax2.set_xlabel('%difference of by-layer \n mean of tracer between runs', fontsize = 15)
    ax2.set_title('preindustrial run - last spinup run', fontsize = 15)
    ax2.invert_yaxis()
    plt.tight_layout()

In [11]:
# BR_TA = np.zeros([40,365*3])
# BR_TA[:,0:365] = BR3_TA- BR1_TA
# BR_TA[:,365:2*365] = BR3_TA - BR2_TA
# PI_TA = np.zeros([40,365*3])
# PI_TA[:,0:365] = PI4_TA- PI1_TA
# PI_TA[:,365:2*365] = PI4_TA - PI2_TA
# PI_TA[:,365*2:3*365] = PI4_TA - PI3rr_TA

# BR_TAperc = np.zeros([40,365*3])
# BR_TAperc[:,0:365] = (BR3_TA- BR1_TA)/BR3_TA *100
# BR_TAperc[:,365:2*365] =( BR3_TA - BR2_TA)/BR3_TA *100
# PI_TAperc = np.zeros([40,365*3])
# PI_TAperc[:,0:365] = (PI4_TA- PI1_TA)/PI4_TA *100
# PI_TAperc[:,365:2*365] = (PI4_TA - PI2_TA)/PI4_TA *100
# PI_TAperc[:,365*2:3*365] = (PI4_TA - PI3rr_TA)/PI4_TA *100

# BR_var = BR_TA
# PI_var = PI_TA
# BR_varperc = BR_TAperc
# PI_varperc = PI_TAperc
# tit = 'TA spinup visualisation'
# tit2 = 'TA by-layer mean differences between \n ultimate and penultimate run'
# pipl = 6 
# brpl = 20


    
# make_spinupplots(PI_var, BR_var, BR_varperc, PI_varperc, tit, tit2, brpl, pipl)

#### THE DIFFERENCES IN THE NON-DIC/TA FIELDS IN THE PI4-PI3RR ARE FREAKISHLY SMALL, BUT THIS MAKES SENSE . AS A REMINDER, TO AVOID DRIFT, WE JUST SET ALL OF THEM TO BE THE SAME AS AT THE BEGINNING OF BR3 AT THE START OF BOTH PI3RR AND PI4. LET'S QUICKLY VISUALISE SOME DIFFERENCES...

In [ ]:

tnc = 'SKOG_1d_20151117_20151216_ptrc_T_20151206-20151206.nc'
pi4dir = '/data/tjarniko/results/BASERUN_EXP/PILA4/PI4/ncs/'
pi3rrdir = '/data/tjarniko/results/BASERUN_EXP/PILA3_rerun/PI3/ncs/'

pi3nc = nc.Dataset(pi3rrdir + tnc)
pi4nc = nc.Dataset(pi4dir + tnc)

pidiff = pi4nc['nitrate'][:] - pi3nc['nitrate'][:]

fact = 0.9
fig = plt.figure(figsize=(12*fact,6.5*fact))

ax0 = fig.add_subplot(1,2,1)
#toba and bute

im = ax0.pcolormesh(pidiff[0,0,:,:], cmap=cm.balance_r)

viz_tools.set_aspect(ax0)

pidiff[0,0,250:260,250:260]

In [ ]:

tnc = 'SKOG_1d_20150401_20150430_ptrc_T_20150415-20150415.nc'
pi4dir = '/data/tjarniko/results/BASERUN_EXP/PILA4/PI4/ncs/'
pi3rrdir = '/data/tjarniko/results/BASERUN_EXP/PILA3_rerun/PI3/ncs/'

pi3nc = nc.Dataset(pi3rrdir + tnc)
pi4nc = nc.Dataset(pi4dir + tnc)

pidiff = pi4nc['diatoms'][:] - pi3nc['diatoms'][:]

fact = 0.9
fig = plt.figure(figsize=(12*fact,6.5*fact))

ax0 = fig.add_subplot(1,2,2)
#toba and bute

im = ax0.pcolormesh(pidiff[0,0,:,:], cmap=cm.balance_r)
plt.title('difference in diatoms - pi4 - pi3rr \n (seems identically 0)')
viz_tools.set_aspect(ax0)

ax1 = fig.add_subplot(1,2,1)
#toba and bute

im = ax1.pcolormesh(pi4nc['diatoms'][0,0,:,:], cmap=cm.balance_r)
plt.title('diatoms - day 15apr, pi4 run')
viz_tools.set_aspect(ax1)
plt.colorbar(im)

print(pidiff[0,0,250:260,250:260])



In [ ]:

tnc = 'SKOG_1d_20150630_20150729_grid_T_20150715-20150715.nc'
pi4dir = '/data/tjarniko/results/BASERUN_EXP/PILA4/PI4/ncs/'
pi3rrdir = '/data/tjarniko/results/BASERUN_EXP/PILA3_rerun/PI3/ncs/'

pi3nc = nc.Dataset(pi3rrdir + tnc)
pi4nc = nc.Dataset(pi4dir + tnc)

pidiff = pi4nc['votemper'][:] - pi3nc['votemper'][:]

fact = 0.9
fig = plt.figure(figsize=(12*fact,6.5*fact))

ax0 = fig.add_subplot(1,2,2)
#toba and bute

im = ax0.pcolormesh(pidiff[0,0,:,:], cmap=cm.balance_r)
plt.title('difference in temp - pi4 - pi3rr \n (seems identically 0)')
viz_tools.set_aspect(ax0)

ax1 = fig.add_subplot(1,2,1)
#toba and bute

im = ax1.pcolormesh(pi4nc['votemper'][0,0,:,:], cmap=cm.balance_r)
plt.title('temperature - day 15jul, pi4 run')
viz_tools.set_aspect(ax1)
plt.colorbar(im)

print(pidiff[0,0,250:260,250:260])

